In [ ]:
### 드랍박스에서 공유링크 가져올 때, 
### recognition.json 에 있는 키값에 일치되는 음성만 
### 공유링크를 뽑자
### 그러려면

In [ ]:
## 김주하 recognition.json에 있는 파일명만 뽑아서 리스트에 저장
import json
import dropbox
from dropbox.exceptions import ApiError
import time

## reconitoin.json 에 있는 파이명만 keyList 에 저장
keyList = []
json_data=open('/Users/joon/Dropbox/recognition_jooha.json').read()
data = json.loads(json_data)

for value in data.keys():
    value = value.split('/')
    keyList.append(value[6])## 파일명만 리스트에 저장

##reconitoin.json에 파일명이 같은게 있으면 공유링크 뽑기, 없으면 pass

db = dropbox.Dropbox('C0w-C7kkDNAAAAAAAAAAE44R8t2689JstWqe2B8W6xMxUZXuSZnYCQ3tF3wqBizQ')
response = db.files_list_folder('/jooha_pydub')
dlList = [] #직접링크 담기 위한 리스트
index = 11885 ## 1873 어디서 부터 할지 ,3773 , 5660, 7574, 9488, 11398,11885
for file in response.entries:
    try:
        if file.name in keyList:## reconitoin.json에 파일명이 같은게 있으면 
            #print(file.name)
            #print(file.path_lower)
            time.sleep(0.1)
            #shared_link_metadata = db.sharing_create_shared_link_with_settings(file.path_lower)
            shared_link_metadata = db.sharing_create_shared_link(file.path_lower) 
            time.sleep(0.1)
            #print (shared_link_metadata.url)
            str = shared_link_metadata.url
            time.sleep(0.1)
            str = str.split('/')
            link = 'https://www.dl.dropboxusercontent.com/s/'
            dlLink = link + str[4]+'/'+str[5]  #직접링크 생성
            time.sleep(0.1)
            dlList.append(dlLink)
            print(repr(index)+' : '+file.name)
            print(dlLink)
            print("-------")
            index +=1

                
    except ApiError as err:
        if err.error.is_shared_link_already_exists():
            print("Link already exists.")
        if err.error.is_path() and err.error.get_path().is_not_found():
            print("File not found.")
        elif err.error.is_settings_error():
            print(err.error.get_settings_error())
        else:
            print(err)
            print('슈바')
            #continue ## 2000 개씩 끊어도 다시 go! --> 소용 없음 2000개에서 끊김


In [ ]:
print(len(dlList))
for i in dlList:
    print(i)

In [ ]:
## 인덱스 0 부터 1872 번까지의 공유링크 txt 생성
f = open("/Users/joon/Desktop/드랍박스_공유링크.txt", 'a')
for i in dlList:
    f.write(i+'\n')
f.close()

In [1]:
## 드랍박스_공유링크.txt 에서 링크 하나씩 읽어 오기 
f = open("/Users/joon/Desktop/드랍박스_공유링크.txt", 'r')## 미국선거와한국정치20161110.0025.wav 1875까지 만듬
idx =0
direct_list = []
while True:
    line = f.readline()
    if not line: break
    #print(line)
    direct_list.append(line)
    idx +=1
f.close()
print(idx)##1873, 3773 # 5660 # 7574 # 9488 # 11398 # 11885 ## 총 1186개 json 기준

11886


In [2]:
### recognition.json 정제할 문장만  뽑아서 디비에 넣을 준비하기

import json
from pprint import pprint

valueList = []

json_data=open('/Users/joon/Dropbox/recognition_jooha.json').read()

data = json.loads(json_data)

for value in data.values():
    #print(value)
    valueList.append(value)

In [3]:
print(len(valueList))

11886


In [5]:
## 직접링크와 정제할 문장 페어 맞춰서 동시에 같이 디비에 넣기(개발 버전/실서비스 버전 구분)
import django
import os
#os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LionSourcing.settings.dev")## 개발 버전
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LionSourcing.settings.prod_aws_eb")##실서비스버전
django.setup()

from main.models import Main

#Main.objects.all().delete() ## 디비 값 삭제

for i in range(len(valueList)):
    Main.objects.create(before_sentences=valueList[i] , direct_links=direct_list[i])
    #break # for 하나로 실험

/Users/joon/anaconda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'direct_links' at row 1")
  result = self._query(query)


In [ ]:
#디비 값 출력 , for 확인
qs = Main.objects.all()
qs = qs[:20]
print(qs)